<a href="https://colab.research.google.com/github/fufu-2345/intelligenceSystem_HW/blob/main/hw_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
import keras
import pandas as pd
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [21]:
data = pd.read_csv("dataset.csv")
print(data.shape)

(400, 11)


In [22]:
data.head()

,gender,age,hypertension,heart_disease,Marriage,work_type,Living_type,avg_glucose,bmi,smoking_status,illness
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [23]:
data.describe()

,age,hypertension,heart_disease,avg_glucose,bmi,illness
count,400.00000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,55.26780,0.180000,0.132500,119.391950,29.481750,0.500000
std,22.51279,0.384669,0.339458,54.377459,6.488354,0.500626
min,0.80000,0.000000,0.000000,56.070000,15.600000,0.000000
25%,44.00000,0.000000,0.000000,80.460000,25.575000,0.000000
50%,59.00000,0.000000,0.000000,97.665000,28.600000,0.500000
75%,74.25000,0.000000,0.000000,144.345000,33.025000,1.000000
max,82.00000,1.000000,1.000000,271.740000,48.900000,1.000000


In [24]:
scaler = StandardScaler()
data[['age', 'hypertension', 'heart_disease', 'avg_glucose', 'bmi', 'illness']] = scaler.fit_transform(data[['age','hypertension','heart_disease','avg_glucose','bmi','illness']])

In [25]:
label_encoder = LabelEncoder()
data['gender'] = label_encoder.fit_transform(data['gender'])
data['Marriage'] = label_encoder.fit_transform(data['Marriage'])
data['work_type'] = label_encoder.fit_transform(data['work_type'])
data['Living_type'] = label_encoder.fit_transform(data['Living_type'])
data['smoking_status'] = label_encoder.fit_transform(data['smoking_status'])

In [26]:
data.head()

,gender,age,hypertension,heart_disease,Marriage,work_type,Living_type,avg_glucose,bmi,smoking_status,illness
0,1,0.521788,-0.468521,2.558744,1,1,1,2.012505,1.098455,1,1.0
1,1,1.099960,-0.468521,2.558744,1,1,0,-0.248059,0.465762,2,1.0
2,0,-0.278759,-0.468521,-0.390817,1,1,1,0.954494,0.758961,3,1.0
3,0,1.055485,2.134375,-0.390817,1,2,0,1.007708,-0.845918,2,1.0
4,1,1.144435,-0.468521,-0.390817,1,1,1,1.230321,-0.074341,1,1.0


In [27]:
y=data['illness']
x=data.drop(columns=['illness'])

In [28]:
y = y.values.reshape(-1, 1)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=42)

In [30]:
# design architecture of model
#model = keras.Sequential()

#model.add(keras.Input(shape=(400,11)))
#model.add(keras.layers.Dense(1, activation="relu"))
#model.add(keras.layers.Dense(1, activation="sigmoid"))

#model.summary()

In [31]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
X_train.shape

(380, 10)

In [33]:

y_train.shape

(380, 1)

In [34]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [35]:
y_train = np.where(y_train == -1, 0, y_train)
y_test = np.where(y_test == -1, 0, y_test)


In [36]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_train_flat = y_train.ravel()  # แปลงเป็น 1 มิติ
classes = np.unique(y_train_flat)  # ดึงค่าคลาสที่มีอยู่

# คำนวณค่า class weight
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_flat)

# แปลงเป็น dictionary โดยใช้ enumerate เพื่อจับคู่ index กับ class
class_weight_dict = {classes[i]: class_weights[i] for i in range(len(classes))}

print(class_weight_dict)  # ดูค่า class weight


{0.0: 1.0, 1.0: 1.0}


In [50]:
# สร้างโมเดล
model = Sequential([
    Flatten(),
    Dense(128, input_dim=10, activation='relu'),
    Dropout(0.35),

    Dense(64, activation='relu'),
    Dropout(0.35),

    Dense(32, activation='relu'),
    Dropout(0.35),

    Dense(16, activation='relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test),
                    batch_size=64, callbacks=[early_stopping], class_weight=class_weight_dict)
model.evaluate(X_test, y_test)

model.save('my_model.keras')

loss, accuracy = model.evaluate(X_test, y_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.5108 - loss: 0.7198 - val_accuracy: 0.7000 - val_loss: 0.6741
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5454 - loss: 0.7046 - val_accuracy: 0.8500 - val_loss: 0.6672
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5012 - loss: 0.7005 - val_accuracy: 0.8000 - val_loss: 0.6586
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5669 - loss: 0.6848 - val_accuracy: 0.8000 - val_loss: 0.6438
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5611 - loss: 0.6776 - val_accuracy: 0.8000 - val_loss: 0.6286
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5878 - loss: 0.6557 - val_accuracy: 0.8500 - val_loss: 0.6075
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6797 - loss: 0.6134 - val_accuracy: 0.8500 - val_loss: 0.5786
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6324 - loss: 0.6250 - val_accuracy: 0.9000 - val_loss:

In [53]:
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy:.2f}%")

Loss: 0.2735774517059326
Accuracy: 0.90%
